<a href="https://colab.research.google.com/github/cyberknopa/ML-in-IS/blob/main/Untitled3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [105]:
import pandas as pd

In [106]:
df_habr = pd.read_csv('randomall.csv')
df_habr.head()

,text,label
0,Актера Алексея Панина выгнали с церемонии пр...,1
1,Генпрокурор США Уильям Барр раскритиковал к...,1
2,Перспектива снижения пенсионного возраста в Ро...,1
3,"Группа «Черкизово» , крупнейший производитель ...",1
4,TikTok -блогер с ником AsapSCIENCE поделился с...,1


In [107]:
import nltk
nltk.download("stopwords")
from nltk.corpus import stopwords
from string import punctuation
russian_stopwords = stopwords.words("russian")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [108]:
# Удаление знаков пунктуации из текста
def remove_punct(text):
    table = {33: ' ', 34: ' ', 35: ' ', 36: ' ', 37: ' ', 38: ' ', 39: ' ', 40: ' ', 41: ' ', 42: ' ', 43: ' ', 44: ' ', 45: ' ', 46: ' ', 47: ' ', 58: ' ', 59: ' ', 60: ' ', 61: ' ', 62: ' ', 63: ' ', 64: ' ', 91: ' ', 92: ' ', 93: ' ', 94: ' ', 95: ' ', 96: ' ', 123: ' ', 124: ' ', 125: ' ', 126: ' '}
    return text.translate(table)

In [109]:
df_habr.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2666 entries, 0 to 2665
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    2665 non-null   object
 1   label   2666 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 41.8+ KB


In [110]:
df_habr.text=df_habr.text.astype(str)

In [111]:
df_habr['clean'] = df_habr['text'].map(lambda x: x.lower())

In [112]:
df_habr['clean'] = df_habr['clean'].map(lambda x: remove_punct(x))
df_habr['clean'] = df_habr['clean'].map(lambda x: x.split(' '))
df_habr['clean'] = df_habr['clean'].map(lambda x: [token for token in x if token not in russian_stopwords\
                                                                  and token != " " \
                                                                  and token.strip() not in punctuation])
df_habr['clean'] = df_habr['clean'].map(lambda x: ' '.join(x))

In [113]:
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(df_habr ['clean'], df_habr ['label'], test_size=0.1, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

In [114]:
display(df_habr)

,text,label,clean
0,Актера Алексея Панина выгнали с церемонии пр...,1,актера алексея панина выгнали церемонии прощан...
1,Генпрокурор США Уильям Барр раскритиковал к...,1,генпрокурор сша уильям барр раскритиковал комп...
2,Перспектива снижения пенсионного возраста в Ро...,1,перспектива снижения пенсионного возраста росс...
3,"Группа «Черкизово» , крупнейший производитель ...",1,группа «черкизово» крупнейший производитель мя...
4,TikTok -блогер с ником AsapSCIENCE поделился с...,1,tiktok блогер ником asapscience поделился подп...
...,...,...,...
2661,Цены на недвижимость в Черногории в обозримом ...,1,цены недвижимость черногории обозримом будущем...
2662,Соответствующее постановление правительства бы...,0,соответствующее постановление правительства из...
2663,Чудовищное известие пришло к нам 6 сентября из...,0,чудовищное известие пришло нам 6 сентября синг...
2664,Открывший стрельбу из окна дома пьяный житель ...,1,открывший стрельбу окна дома пьяный житель юго...


In [115]:
from sklearn.pipeline import Pipeline
# pipeline позволяет объединить в один блок трансформер и модель, что упрощает написание кода и улучшает его читаемость
from sklearn.feature_extraction.text import TfidfVectorizer
# TfidfVectorizer преобразует тексты в числовые вектора, отражающие важность использования каждого слова из некоторого набора слов (количество слов набора определяет размерность вектора) в каждом тексте
from sklearn.linear_model import SGDClassifier
from sklearn.neighbors import KNeighborsClassifier
# линейный классификатор и классификатор методом ближайших соседей
from sklearn import metrics
# набор метрик для оценки качества модели
from sklearn.model_selection import GridSearchCV
# модуль поиска по сетке параметров

In [116]:
sgd_ppl_clf = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('sgd_clf', SGDClassifier(random_state=42))])
knb_ppl_clf = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('knb_clf', KNeighborsClassifier(n_neighbors=10))])
sgd_ppl_clf.fit(X_train, y_train)
knb_ppl_clf.fit(X_train, y_train)

Pipeline(steps=[('tfidf', TfidfVectorizer()),
                ('knb_clf', KNeighborsClassifier(n_neighbors=10))])

In [117]:
sgd_ppl_clf.fit(X_train, y_train)

Pipeline(steps=[('tfidf', TfidfVectorizer()),
                ('sgd_clf', SGDClassifier(random_state=42))])

In [118]:
predicted_sgd = sgd_ppl_clf.predict(X_test)
print(metrics.classification_report(predicted_sgd, y_test))

              precision    recall  f1-score   support

           0       0.95      0.98      0.97       215
           1       0.98      0.96      0.97       265

    accuracy                           0.97       480
   macro avg       0.97      0.97      0.97       480
weighted avg       0.97      0.97      0.97       480



In [119]:
sgd_ppl_clf = Pipeline([
    ('tfidf', TfidfVectorizer(ngram_range=(1, 4), strip_accents='unicode')),
    ('sgd_clf', SGDClassifier(penalty='elasticnet', class_weight='balanced', random_state=42))
])

In [120]:
sgd_ppl_clf.fit(X_train, y_train)

Pipeline(steps=[('tfidf',
                 TfidfVectorizer(ngram_range=(1, 4), strip_accents='unicode')),
                ('sgd_clf',
                 SGDClassifier(class_weight='balanced', penalty='elasticnet',
                               random_state=42))])

In [121]:
predicted_sgd = sgd_ppl_clf.predict(X_test)
print(metrics.classification_report(predicted_sgd, y_test))

              precision    recall  f1-score   support

           0       0.90      0.98      0.94       200
           1       0.99      0.92      0.95       280

    accuracy                           0.95       480
   macro avg       0.94      0.95      0.94       480
weighted avg       0.95      0.95      0.95       480



In [122]:
knb_ppl_clf.fit(X_train, y_train)

Pipeline(steps=[('tfidf', TfidfVectorizer()),
                ('knb_clf', KNeighborsClassifier(n_neighbors=10))])

In [123]:
predicted_sgd = knb_ppl_clf.predict(X_test)
print(metrics.classification_report(predicted_sgd, y_test))

              precision    recall  f1-score   support

           0       0.78      0.93      0.85       184
           1       0.95      0.84      0.89       296

    accuracy                           0.88       480
   macro avg       0.87      0.89      0.87       480
weighted avg       0.89      0.88      0.88       480



In [124]:
predicted_sgd_val = sgd_ppl_clf.predict(X_valid)
print(metrics.classification_report(predicted_sgd_val, y_valid))

              precision    recall  f1-score   support

           0       0.92      1.00      0.96       112
           1       1.00      0.94      0.97       155

    accuracy                           0.96       267
   macro avg       0.96      0.97      0.96       267
weighted avg       0.97      0.96      0.96       267



In [143]:
sample_text=('В сотрудничестве с ведущими немецкими языковедами и филологами, федеральным министерством образования и научных исследований Германии, был представлен обновленный политкорректный вариант школьного учебника по немецкому языку. Особое внимание в пособии уделено разделу «Корректные извинения перед гостями из Африки». Кроме форм правильного обращения к иностранцам, в книге можно ознакомиться с кулинарными предпочтениями народов, тенденциями моды и особенностями религиозных обычаев. «Многие гости из стран Африки стали полноценными членами немецкого социума. Если мы не научимся сами, и не научим своих детей правильно с ними взаимодействовать, может случиться серьезный конфликт, который никому не нужен», — сказала глава министерства образования ФРГ Аня Карличек.=')

In [144]:
predicted_sgd = sgd_ppl_clf.predict([sample_text])

In [145]:
print(predicted_sgd)

[0]
